# Power management
Uses https://github.com/danjulio/MPPT-Solar-Charger

In [ ]:
%websocketconnect 192.168.1.130:8266 --password horsetoe

In [ ]:
global wdt_counter
wdt_counter = 0

In [ ]:
#%serialconnect /dev/tty.USB0 115200

In [ ]:
%meminfo

![Pin details](pin_details.png)

In [ ]:
import machine

SDA = machine pin 4 = D2

SCL = machine pin 5 = D1

In [ ]:
i2c = machine.I2C(sda=machine.Pin(4), scl=machine.Pin(5))

In [ ]:
devices = i2c.scan()

In [ ]:
if len(devices) == 0:
  print("No i2c device !")
else:
  print('i2c devices found:', len(devices))

  for device in devices:  
    print("Decimal address: ", device, " | Hexa address: ", hex(device))

### The solar charger is at address 18

In [ ]:
mppt_i2c_address = 18

In [ ]:
i2c.readfrom(mppt_i2c_address, 4) 

In [ ]:
global wdt_counter
wdt_counter = 0

# I2c register constants

From https://github.com/danjulio/MPPT-Solar-Charger/blob/master/arduino/mpptChg/mpptChg.h

```
#RO values (16-bits)
MPPT_CHG_REG_ID = 0
MPPT_CHG_STATUS = 2
MPPT_CHG_BUCK = 4
MPPT_CHG_VS = 6
MPPT_CHG_IS = 8
MPPT_CHG_VB = 10
MPPT_CHG_IB = 12
MPPT_CHG_IC = 14
MPPT_CHG_INT_T = 16
MPPT_CHG_EXT_T = 18
MPPT_CHG_VM = 20
MPPT_CHG_TH = 22
#RW Parameters (16-bits)
MPPT_CHG_BUCK_TH = 24
MPPT_CHG_FLOAT_TH = 26
MPPT_CHG_PWROFF = 28
MPPT_CHG_PWRON = 30
#Watchdog registers (8-bits)
MPPT_WD_EN = 33
MPPT_WD_COUNT = 35
MPPT_WD_PWROFF = 36
```

```
//
// ID Register bit masks
//
#define MPPT_CHG_ID_BRD_ID_MASK  0xF000
#define MPPT_CHG_ID_MAJ_REV_MASK 0x00F0
#define MPPT_CHG_ID_MIN_REV_MASK 0x000F
```

```
#Status Register bit masks

MPPT_CHG_STATUS_HW_WD_MASK = 0x8000
MPPT_CHG_STATUS_SW_WD_MASK = 0x4000
MPPT_CHG_STATUS_BAD_BATT_MASK = 0x2000
MPPT_CHG_STATUS_EXT_MISS_MASK = 0x1000
MPPT_CHG_STATUS_WD_RUN_MASK = 0x0100
MPPT_CHG_STATUS_PWR_EN_MASK = 0x0080
MPPT_CHG_STATUS_ALERT_MASK = 0x0040
MPPT_CHG_STATUS_PCTRL_MASK = 0x0020
MPPT_CHG_STATUS_T_LIM_MASK = 0x0010
MPPT_CHG_STATUS_NIGHT_MASK = 0x0008
MPPT_CHG_STATUS_CHG_ST_MASK = 0x0007
```

```
//
// Status Register Charge States
//
#define MPPT_CHG_ST_NIGHT  0
#define MPPT_CHG_ST_IDLE   1
#define MPPT_CHG_ST_VSRCV  2
#define MPPT_CHG_ST_SCAN   3
#define MPPT_CHG_ST_BULK   4
#define MPPT_CHG_ST_ABSORB 5
#define MPPT_CHG_ST_FLOAT  6

//
// Buck Status bit masks
//
#define MPPT_CHG_BUCK_PWM_MASK  0xFF00
#define MPPT_CHG_BUCK_LIM2_MASK 0x0002
#define MPPT_CHG_BUCK_LIM1_MASK 0x0001

//
// Watchdog enable register value
//
#define MPPT_CHG_WD_ENABLE 0xEA
```

### Set up an array to take data

In [ ]:
mppt_id_data = bytearray(2)

#### Read from the charge register ID

In [ ]:
MPPT_CHG_REG_ID = 0

In [ ]:
i2c.readfrom_mem_into(mppt_i2c_address, MPPT_CHG_REG_ID, mppt_id_data)

In [ ]:
print(mppt_id_data)

In [ ]:
from struct import unpack
unpack('>h', mppt_id_data)[0]

In [ ]:
global wdt_counter
wdt_counter = 0

### Get the charge status

In [ ]:
mppt_chg_status = bytearray(2)

In [ ]:
MPPT_CHG_STATUS = 2

In [ ]:
i2c.readfrom_mem_into(mppt_i2c_address, MPPT_CHG_STATUS, mppt_chg_status)

In [ ]:
print(mppt_chg_status)

In [ ]:
from struct import unpack
unpack('>h', mppt_chg_status)[0]

### Get the charge state

```
// Status Register Charge States
//
#define MPPT_CHG_ST_NIGHT  0
#define MPPT_CHG_ST_IDLE   1
#define MPPT_CHG_ST_VSRCV  2
#define MPPT_CHG_ST_SCAN   3
#define MPPT_CHG_ST_BULK   4
#define MPPT_CHG_ST_ABSORB 5
#define MPPT_CHG_ST_FLOAT  6
```

In [ ]:
MPPT_CHG_STATUS_CHG_ST_MASK = 0x0007

In [ ]:
status_index = unpack('>h', mppt_chg_status)[0] & MPPT_CHG_STATUS_CHG_ST_MASK

In [ ]:
status_index

### Get the battery voltage

In [ ]:
global wdt_counter
wdt_counter = 0

In [ ]:
mppt_chg_vb = bytearray(2)

In [ ]:
MPPT_CHG_VB = 10

In [ ]:
i2c.readfrom_mem_into(mppt_i2c_address, MPPT_CHG_VB, mppt_chg_vb)

In [ ]:
mppt_chg_vb

In [ ]:
from struct import unpack
unpack('>h', mppt_chg_vb)[0] / 1000

In [ ]:
def getBatteryStatus():
    import struct
    import machine
    import utime
        
    i2c = machine.I2C(sda=machine.Pin(4), scl=machine.Pin(5))
    mppt_i2c_address = 18
    MPPT_CHG_STATUS = 2 # Charge status
    MPPT_CHG_STATUS_CHG_ST_MASK = 0x0007
    MPPT_CHG_VS = 6 # Supply voltage
    MPPT_CHG_IS = 8 # Supply current
    MPPT_CHG_VB = 10 # Battery voltage
    MPPT_CHG_IB = 12 # Battery current
    MPPT_CHG_EXT_T = 18 # External temperature
    
    mppt_chg_status = bytearray(2)
    i2c.readfrom_mem_into(mppt_i2c_address, MPPT_CHG_STATUS, mppt_chg_status)
    status_index = struct.unpack('>h', mppt_chg_status)[0] & MPPT_CHG_STATUS_CHG_ST_MASK
    
    mppt_chg_vs = bytearray(2)
    mppt_chg_is = bytearray(2)
    i2c.readfrom_mem_into(mppt_i2c_address, MPPT_CHG_VS, mppt_chg_vs)
    i2c.readfrom_mem_into(mppt_i2c_address, MPPT_CHG_IS, mppt_chg_is)
    solar_voltage = struct.unpack('>h', mppt_chg_vs)[0]
    solar_current = struct.unpack('>h', mppt_chg_is)[0]
    solar_watts = (solar_voltage * solar_current) / 1000000.0 # W = mA * mA / 1E6
    
    mppt_chg_vb = bytearray(2)
    i2c.readfrom_mem_into(mppt_i2c_address, MPPT_CHG_VB, mppt_chg_vb)
    battery_voltage = struct.unpack('>h', mppt_chg_vb)[0] / 1000
    
    mppt_chg_ib = bytearray(2)
    i2c.readfrom_mem_into(mppt_i2c_address, MPPT_CHG_IB, mppt_chg_ib)
    battery_amperage = struct.unpack('>h', mppt_chg_ib)[0] / 1000
    
    mppt_chg_ext_t = bytearray(2)
    i2c.readfrom_mem_into(mppt_i2c_address, MPPT_CHG_EXT_T, mppt_chg_ext_t)
    external_temperature = struct.unpack('>h', mppt_chg_ext_t)[0] / 10
    
    return {"status_index" : status_index, "solar_watts" : solar_watts,
            "battery_voltage" : battery_voltage, "battery_amperage" : battery_amperage,
            "external_temperature" : external_temperature,
            "power_time_date" : list(utime.localtime())}

In [ ]:
getBatteryStatus()

In [ ]:
import send_data
send_data.postData(getBatteryStatus(), url="http://192.168.1.151:9494/power")

In [ ]:
global wdt_counter
wdt_counter = 0

```
// Status Register Charge States
//
#define MPPT_CHG_ST_NIGHT  0
#define MPPT_CHG_ST_IDLE   1
#define MPPT_CHG_ST_VSRCV  2
#define MPPT_CHG_ST_SCAN   3
#define MPPT_CHG_ST_BULK   4
#define MPPT_CHG_ST_ABSORB 5
#define MPPT_CHG_ST_FLOAT  6
```

In [ ]:
power_script = """def getBatteryStatus():
    import struct
    import machine
    import utime
        
    i2c = machine.I2C(sda=machine.Pin(4), scl=machine.Pin(5))
    mppt_i2c_address = 18
    MPPT_CHG_STATUS = 2 # Charge status
    MPPT_CHG_STATUS_CHG_ST_MASK = 0x0007
    MPPT_CHG_VS = 6 # Supply voltage
    MPPT_CHG_IS = 8 # Supply current
    MPPT_CHG_VB = 10 # Battery voltage
    MPPT_CHG_IB = 12 # Battery current
    MPPT_CHG_EXT_T = 18 # External temperature
    
    mppt_chg_status = bytearray(2)
    i2c.readfrom_mem_into(mppt_i2c_address, MPPT_CHG_STATUS, mppt_chg_status)
    status_index = struct.unpack('>h', mppt_chg_status)[0] & MPPT_CHG_STATUS_CHG_ST_MASK
    
    mppt_chg_vs = bytearray(2)
    mppt_chg_is = bytearray(2)
    i2c.readfrom_mem_into(mppt_i2c_address, MPPT_CHG_VS, mppt_chg_vs)
    i2c.readfrom_mem_into(mppt_i2c_address, MPPT_CHG_IS, mppt_chg_is)
    solar_voltage = struct.unpack('>h', mppt_chg_vs)[0]
    solar_current = struct.unpack('>h', mppt_chg_is)[0]
    solar_watts = (solar_voltage * solar_current) / 1000000.0 # W = mA * mA / 1E6
    
    mppt_chg_vb = bytearray(2)
    i2c.readfrom_mem_into(mppt_i2c_address, MPPT_CHG_VB, mppt_chg_vb)
    battery_voltage = struct.unpack('>h', mppt_chg_vb)[0] / 1000
    
    mppt_chg_ib = bytearray(2)
    i2c.readfrom_mem_into(mppt_i2c_address, MPPT_CHG_IB, mppt_chg_ib)
    battery_amperage = struct.unpack('>h', mppt_chg_ib)[0] / 1000
    
    mppt_chg_ext_t = bytearray(2)
    i2c.readfrom_mem_into(mppt_i2c_address, MPPT_CHG_EXT_T, mppt_chg_ext_t)
    external_temperature = struct.unpack('>h', mppt_chg_ext_t)[0] / 10
    
    return {"status_index" : status_index, "solar_watts" : solar_watts,
            "battery_voltage" : battery_voltage, "battery_amperage" : battery_amperage,
            "external_temperature" : external_temperature,
            "power_time_date" : list(utime.localtime())}"""

In [ ]:
import os
print(os.listdir())

In [ ]:
with open('power_status.py') as f:
    print(f.read())

In [ ]:
os.remove('power_status.py')

In [ ]:
with open('power_status.txt', 'w') as f:
    f.write(power_script)

### Double check the write

In [ ]:
with open('power_status.txt') as f:
    print(f.read())

## Move temp file to `boot.py`

In [ ]:
os.rename('power_status.txt', 'power_status.py')

In [ ]:
%disconnect